In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用Vertex AI私有端点开始

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/prediction/get_started_with_vertex_private_endpoints.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fprediction%2Fget_started_with_vertex_private_endpoints.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/prediction/get_started_with_vertex_private_endpoints.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在工作台中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/prediction/get_started_with_vertex_private_endpoints.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

## 概述

本教程演示了如何使用Vertex AI SDK来创建和使用`Vertex AI Endpoint`资源来提供模型服务。`Vertex AI Private Endpoints`。 `私有端点`提供了客户端和服务器之间点对点的网络gRPC通信（即，局域网），在同一网络内。这消除了使用HTTP协议（即，互联网）的公共端点的网络切换和路由的开销。

了解更多关于[Private Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints)。

### 目标

在本教程中，您将学习如何使用`Vertex AI私有端点`资源。

本教程使用以下Google Cloud ML服务和资源：

- `Vertex AI端点`
- `Vertex AI模型`
- `Vertex AI预测`

执行的步骤包括：

- 创建一个`私有端点`资源。
- 配置VPC对等连接。
- 配置一个`模型`资源的服务二进制文件，以便部署到`私有端点`资源。
- 将一个`模型`资源部署到`私有端点`资源。
- 向`私有端点`发送预测请求。

#### `私有端点`设置与`公共端点`有何不同

- 启用两个额外的API：服务网络和云DNS。
- 将计算管理员网络角色添加到您的（默认）服务帐号中。
- 发出两个gcloud命令以设置VPC对等连接以供您的服务帐号使用。
- 目前还没有SDK支持，因此私有端点是使用GAPIC客户端创建的，并且对于对等网络有一个额外参数。
- 要发送请求，您不能使用SDK/GAPIC，因为它们会进行HTTP互联网请求。相反，您可以使用curl发送对等请求。

数据集

本教程使用了来自TensorFlow Hub的一个预训练图像分类模型，该模型是在ImageNet数据集上训练的。

了解更多关于[ResNet V2预训练模型](https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5)。

成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

开始吧.

### GPU运行时

如果您有此选项，请确保在GPU运行时中运行此笔记本。在Colab中，选择 **运行时 > 更改运行时类型 > GPU**。

### 为Python安装Vertex AI SDK和其他必需的包

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform
! pip3 install --upgrade --quiet google-cloud-pipeline-components
! pip3 install tensorflow-hub==0.16.1
! pip3 install --upgrade --quiet tensorflow==2.15.1

重启运行时（仅限于Colab）

要使用新安装的软件包，您必须在Google Colab上重启运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核即将重新启动。请等待它完成后再继续下一步。⚠️</b>
</div>

### 在谷歌Colab上验证您的笔记本环境

在谷歌Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息并初始化Python版的Vertex AI SDK

要开始使用Vertex AI，您必须拥有现有的Google Cloud项目并[启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

初始化用于Python的Vertex AI SDK

为您的项目和相应的存储桶初始化用于Python的Vertex AI SDK。

In [ ]:
import google.cloud.aiplatform as aip

aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置项目（仅限协作）

In [ ]:
if "google.colab" in sys.modules:
    ! gcloud config set project $PROJECT_ID

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常数

In [ ]:
import os

import tensorflow as tf
import tensorflow_hub as hub

设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`DEPLOY_GPU / DEPLOY_NGPU`设置为使用支持GPU的容器映像以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个支持GPU的容器映像，其中为每个VM分配了 4个 Nvidia Telsa K80 GPU，您可以指定：

（aip.gapic.AcceleratorType.NVIDIA_TESLA_K80，4）

否则，请指定（无，无）以使用在CPU上运行的容器映像。

了解更多关于您位置的[硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

In [ ]:
DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设定预构建的容器

设置用于预测的预构建Docker容器镜像。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TF = "2.5".replace(".", "-")

if DEPLOY_GPU:
    DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
else:
    DEPLOY_VERSION = "tf2-cpu.{}".format(TF)


DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个 vCPU 3.75GB 的内存。
     - `n1-highmem`：每个 vCPU 6.5GB 的内存。
     - `n1-highcpu`：每个 vCPU 0.9GB 的内存。
 - `vCPUs`：数量为 \[2, 4, 8, 16, 32, 64, 96 \]

*注意：您也可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持 GPU*。

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

## 从 TensorFlow Hub 获取预训练模型

为了演示目的，本教程使用了来自 TensorFlow Hub (TFHub) 的预训练模型，然后将其上传到 `Vertex AI Model` 资源中。一旦您拥有一个 `Vertex AI Model` 资源，该模型就可以部署到 `Vertex AI Private Endpoint` 资源上。

### 下载预训练模型

首先，您从 TensorFlow Hub 下载预训练模型。该模型被下载为一个 TF.Keras 层。为了完成模型， 在这个例子中，您使用下载的 TFHub 模型创建一个 `Sequential()` 模型，并指定模型的输入形状。

In [ ]:
tfhub_model = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model.build([None, 224, 224, 3])

tfhub_model.summary()

### 保存模型工件

此时，模型已存储在内存中。接下来，将模型工件保存到云存储位置。

In [ ]:
MODEL_DIR = BUCKET_URI + "/model"
tfhub_model.save(MODEL_DIR)

## 上传模型以提供服务

接下来，您将从自定义作业中上传您的TF.Keras模型到Vertex `Model`服务，这将为您的自定义模型创建一个Vertex `Model`资源。在上传过程中，您需要定义一个提供函数，将数据转换为模型期望的格式。如果将编码数据发送到Vertex AI，您的提供函数会确保在将数据作为输入传递给模型之前，在模型服务器上对数据进行解码。

### 提供函数如何工作

当您向在线预测服务器发送请求时，请求将由HTTP服务器接收。HTTP服务器从HTTP请求内容体中提取预测请求。提取的预测请求将被转发到提供函数。对于谷歌预构建的预测容器，请求内容作为`tf.string`传递给提供函数。

提供函数由两部分组成：

- `预处理函数`：
  - 将输入 (`tf.string`) 转换为基础模型的输入形状和数据类型 (动态图)。
  - 执行与训练基础模型期间相同的数据预处理 -- 如，归一化、缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序所期望的格式 -- 如，压缩输出。
  - 为接收应用程序打包输出 -- 如，添加标题，创建JSON对象等。

预处理和后处理函数都转换为与模型融合的静态图。基础模型的输出传递到后处理函数。后处理函数将转换/打包后的输出传递回HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

在构建TF.Keras模型的提供函数时，您需要考虑一个问题是它们作为静态图运行。这意味着，您不能使用需要动态图的TF图操作。如果这样做，您将在提供函数的编译过程中遇到错误，该错误将指示您正在使用不受支持的EagerTensor。

### 为图像数据提供服务功能

#### 预处理

为了将图像传递给预测服务，您将压缩的（例如JPEG）图像字节编码为base 64 -- 这样在通过网络传输二进制数据时可以确保内容不被修改。由于这个部署的模型期望输入数据是原始（未压缩的）字节，您需要确保将base 64编码的数据转换回原始字节，然后经过预处理以匹配模型的输入要求，然后将其作为输入传递给部署的模型。

为了解决这个问题，您需要定义一个服务函数（`serving_fn`）并将其附加到模型作为预处理步骤。添加`@tf.function`装饰器，以便服务函数与底层模型融合（而不是在CPU上游上）。

当您发送预测或解释请求时，请求的内容被解码为一个Tensorflow字符串（`tf.string`），然后传递给服务函数（`serving_fn`）。服务函数将`tf.string`预处理为原始（未压缩的）numpy字节（`preprocess_fn`）以匹配模型的输入要求：

- `io.decode_jpeg` - 解压缩返回一个带有三个通道（RGB）的Tensorflow张量的JPG图像。
- `image.convert_image_dtype` - 将整数像素值更改为float 32，并在0和1之间重新调整像素数据。
- `image.resize` - 将图像调整大小以匹配模型的输入形状。

此时，数据可以通过具体函数传递给模型（`m_call`）。服务函数是一个静态图，而模型是一个动态图。具体函数执行从服务函数到模型的输入数据的编组任务，并将从模型到服务函数的预测结果进行编组。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(224, 224))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(tfhub_model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(tfhub_model, MODEL_DIR, signatures={"serving_default": serving_fn})

获取serving函数的签名

您可以通过重新加载模型到内存，并查询每个层对应的签名来获取模型的输入和输出层的签名。

对于您的目的，您需要serving函数的签名。原因是当您将数据作为HTTP请求数据包发送进行预测时，图像数据是base64编码的，而您的TF.Keras模型需要numpy输入。您的serving函数会将数据从base64转换为numpy数组。

在进行预测请求时，您需要将请求路由到serving函数而不是模型，因此您需要知道serving函数的输入层名称--这将在您后续进行预测请求时使用。

In [ ]:
loaded = tf.saved_model.load(MODEL_DIR)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

将TensorFlow Hub模型上传到“Vertex AI Model”资源

最后，将TFHub模型的模型工件上传到“Vertex AI Model”资源中。

In [ ]:
model = aip.Model.upload(
    display_name="example",
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
)

print(model)

## 设置VPC对等网络

要使用`私有端点`，您需要在您的项目和托管运行您的模型的`Vertex AI Prediction`服务项目之间设置一个VPC对等网络。这消除了网络流量中的额外跳数，并允许使用高效的gRPC协议。

了解更多关于[VPC对等](https://cloud.google.com/vertex-ai/docs/general/vpc-peering)。

**重要提示：每个项目只能设置一次VPC对等到servicenetworking.googleapis.com。**

### 为默认网络创建VPC对等连接

为了简单起见，设置VPC对等连接到默认网络。您可以为您的项目创建一个不同的网络。

如果您设置了与任何其他网络的VPC对等连接，请确保该网络已经存在并且您的虚拟机正在该网络上运行。

In [ ]:
# This is for display only; you can name the range anything.
PEERING_RANGE_NAME = "vertex-ai-prediction-peering-range"
NETWORK = "default"

# NOTE: `prefix-length=16` means a CIDR block with mask /16 will be
# reserved for use by Google services, such as Vertex AI.
! gcloud compute addresses create $PEERING_RANGE_NAME \
  --global \
  --prefix-length=16 \
  --description="peering range for Google service" \
  --network=$NETWORK \
  --purpose=VPC_PEERING

### 创建 VPC 连接

接下来，创建 VPC 对等连接。

*注意:* 如果你遇到权限被拒绝的情况，可能是因为你的默认服务账号没有设置必要的角色 'Compute Network Admin'。在云控制台中，按照以下步骤操作。

1. 进入 `IAM & Admin`。
2. 找到你的服务账号。
3. 点击编辑图标。
4. 选择 `添加另一个角色`。
5. 输入 'Compute Network Admin'。
6. 点击 `保存`。

In [ ]:
! gcloud services vpc-peerings connect \
  --service=servicenetworking.googleapis.com \
  --network=$NETWORK \
  --ranges=$PEERING_RANGE_NAME \
  --project=$PROJECT_ID

检查您对等连接的状态。

In [ ]:
! gcloud compute networks peerings list --network $NETWORK

构建完整的网络名称

在随后为VPC对等连接创建`私有端点`资源时，您需要拥有完整的网络资源名称。

In [ ]:
project_number = model.resource_name.split("/")[1]
print(project_number)

full_network_name = f"projects/{project_number}/global/networks/{NETWORK}"

## 创建 `Endpoint` 资源

您可以使用 `PrivateEndpoint.create()` 方法来创建一个 `Endpoint` 资源。

在本例中，指定了以下参数：

- `display_name`：`Private Endpoint` 资源的人类可读名称。
- `network`：VPC peering 的完整网络资源名称。

了解有关[Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)的更多信息。

In [ ]:
if not os.getenv("IS_TESTING"):
    endpoint = aip.PrivateEndpoint.create(
        display_name="private", network=full_network_name
    )

获取`Endpoint`资源的详细信息

您可以使用属性`gca_resource`获取`Endpoint`对象的基本详细信息。

In [ ]:
if not os.getenv("IS_TESTING"):
    endpoint.gca_resource

将`Model`资源部署到`Endpoint`资源。

您可以将一个或多个`Vertex AI Model`资源实例部署到同一个端点。每个部署的`Vertex AI Model`资源将拥有自己的用于服务二进制文件的部署容器。

*注意：* 对于此示例，在上传模型工件到`Vertex AI Model`资源的先前步骤中，您指定了TFHub模型的部署容器。

### 部署单个`Endpoint`资源

在下一个示例中，您将一个`Vertex AI Model`资源部署到一个`Vertex AI Endpoint`资源。要进行部署，您需要指定以下额外的配置设置：

- 机器类型。
- （如果有的话）GPU的类型和数量。
- 静态、手动或自动扩展的VM实例。

对于`私有端点`，您只能部署一个模型。因此，没有流量分割。

在此示例中，您将以最少的参数部署模型，如下所示：

- `model`：`Model`资源。
- `deployed_model_displayed_name`：部署的模型实例的可读名称。
- `machine_type`：每个VM实例的机器类型。

由于需要分配资源，这可能需要几分钟时间。

In [ ]:
from datetime import datetime

# Get the current timestamp
current_timestamp = datetime.now()

# Convert the timestamp to a string
TIMESTAMP = current_timestamp.strftime("%Y_%m_%d_%H_%M_%S")

if not os.getenv("IS_TESTING"):
    response = endpoint.deploy(
        model=model,
        deployed_model_display_name="example_" + TIMESTAMP,
        machine_type=DEPLOY_COMPUTE,
    )

    print(endpoint)

获取部署模型的信息

您可以从 `Endpoint` 资源配置数据 `gca_resource.deployed_models` 中获取部署模型的部署设置。在这个示例中，只部署了一个模型--因此引用下标 `[0]`。

In [ ]:
if not os.getenv("IS_TESTING"):
    print(endpoint.gca_resource.deployed_models[0])

### 为预测准备测试数据

接下来，您将把一个压缩的JPEG图像加载到内存中，然后对其进行base64编码。为了演示目的，您将使用来自花卉数据集的一张图片。

In [ ]:
! gsutil cp gs://cloud-ml-data/img/flower_photos/daisy/100080576_f52e8ee070_n.jpg test.jpg

In [ ]:
import base64

with open("test.jpg", "rb") as f:
    data = f.read()
b64str = base64.b64encode(data).decode("utf-8")

### 进行预测

现在您的`Model`资源已部署到一个`Endpoint`资源中，您可以通过向Endpoint资源发送预测请求来进行在线预测。

#### 请求

由于`私有Endpoint`将阻止来自公共HTTP请求的请求，因此您将使用`curl`向私有URI发送请求。

首先，您将构建请求作为JSON文件。为了将测试数据传递给预测服务，您需要将字节编码为base64 -- 这样可以使内容在通过网络传输二进制数据时免受修改。

每个实例的格式为：

    { serving_input: { 'b64': base64编码的字节 } }

由于服务二进制可以包含多个项目（实例），请将您单个的测试项目发送为一个测试项目列表。

In [ ]:
import json

with open("instances.json", "w") as f:
    f.write(json.dumps({"instances": [{serving_input: {"b64": b64str}}]}))

使用SDK发出预测请求

接下来，使用Python的Vertex AI SDK发出预测请求。如果笔记本在VPC网络中，则该请求将被执行。因此，在谷歌colab中无法执行预测请求，因为它不是VPC网络的一部分。

In [ ]:
instances = [{serving_input: {"b64": b64str}}]

IS_COLAB = "google.colab" in sys.modules

if not os.getenv("IS_TESTING"):
    if not IS_COLAB:
        prediction = endpoint.predict(instances=instances)
        print(prediction)

### 从 `Endpoint` 资源中取消部署 `Model` 资源

当 `Model` 资源部署到 `Endpoint` 资源时，部署的 `Model` 资源实例会被分配一个ID -- 通常被称为部署的模型ID。

您可以使用 `undeploy()` 方法取消部署特定的 `Model` 资源实例，参数如下：

- `deployed_model_id`：分配给部署模型的ID。

In [ ]:
if not os.getenv("IS_TESTING"):
    deployed_model_id = endpoint.gca_resource.deployed_models[0].id
    print(deployed_model_id)

    endpoint.undeploy(deployed_model_id)

最终，您可以使用`undeploy_all()`方法从`Endpoint`资源中取消部署所有`Model`实例。

In [ ]:
if not os.getenv("IS_TESTING"):
    try:
        endpoint.undeploy_all()
    except IndexError as e:
        print("Exception cought: ", e)

### 删除一个 `Endpoint` 资源

如果 `Endpoint` 资源没有部署模型，您可以使用 `delete()` 方法来删除该 `Endpoint` 资源。

In [ ]:
if not os.getenv("IS_TESTING"):
    endpoint.delete()

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除您用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除您在本教程中创建的各个资源。

In [ ]:
delete_bucket = False
delete_model = True
delete_generated_files = False

if delete_model:
    try:
        model.delete()
    except Exception as e:
        print(e)

if delete_bucket:
    ! gsutil rm -rf {BUCKET_URI}

if delete_generated_files:
    ! rm -rf "test.jpg" "instances.json"